In [1]:
import numpy as np
import pandas as pd
from gensim.models import FastText

In [2]:
DATA_IN_PATH = '../Embedding/embedding_data/'
model = FastText.load(DATA_IN_PATH + 'FastText_embedding.model').wv
# FastText 모델 불러오기

In [3]:
label_data = pd.read_csv('../data/naverReview_label.csv')
train_data = pd.read_csv('../data/preprocess/naverReview_preprocess_Okt.csv', encoding = 'UTF8')
#필요한 라벨, 트레인 데이터 불러오기

In [4]:
new_data = pd.concat([train_data, label_data], axis=1)
#nan제거를 위해 합치기

In [5]:
new_data

,context,label
0,아 더빙 진짜 짜증나다 목소리,0
1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍다 않다,1
2,너 무재 밓었 다그 래서 보다 추천 한 다,0
3,교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정,0
4,사이 몬페 그 의 익살스럽다 연기 가 돋보이다 영화 스파이더맨 에서 늙다 보이다 하...,1
...,...,...
199987,오랜 만 에 평점 로 기다 하다 ㅋㅋ 킹왕짱 쌈뽕 한 영화 를 만나다 강렬하다 육 쾌함,1
199988,의지 박약 들 이나 하다 탈영 은 일단 주인공 김대희 닮다 이등병 찌다 따다,0
199989,그림 도 좋다 완성 도도 높다 보다 내내 불안하다 만들다,0
199990,절대 보다 서다 안 되다 영화 재미 도 없다 기분 만 잡 치고 하다 세트 장 에서 ...,0


In [6]:
new_data2 = new_data.dropna()
#nan 제거

In [7]:
labels = list(new_data2['label'])
contexts = list(new_data2['context'])
#라벨 리뷰 분리
sentences = []
for context in contexts :
    sentences.append(str(context).split())

In [8]:
def get_feature(words, model, num_features) :
    feature_vector = np.zeros((num_features), dtype = np.float32)
    
    num_words = 0
    index2word_set = set(model.wv.index2word)
    
    for w in words :
        if w in index2word_set :
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [9]:
def get_dataset(reviews, model, num_features) :
    dataset = list()
    
    for s in reviews :
        dataset.append(get_feature(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [10]:
num_features = 100
test_data_vecs = get_dataset(sentences, model, num_features)

<ipython-input-8-c82fc869ee2a>:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model.wv.index2word)
<ipython-input-8-c82fc869ee2a>:11: RuntimeWarning: invalid value encountered in true_divide
  feature_vector = np.divide(feature_vector, num_words)


In [ ]:
test_data = test_data_vecs.tolist()
test_data_df = pd.DataFrame({'context' : test_data})
label_df = pd.DataFrame({'label': labels})
#nan 제거를 위해 각각 df 화

In [12]:
new_df = pd.concat([test_data_df, label_df], axis=1)
#nan 제거를 위해 병합

In [13]:
nan_index = []
for index in range(198315) :
    npArray = np.asarray(new_df['context'].iloc[index], dtype=np.float32)
    if np.isnan(npArray).any() :
        nan_index.append(index)
        
print(nan_index)
#nan 찾는 함수

[28, 79, 151, 169, 414, 1150, 1349, 1575, 1715, 2328, 2352, 2440, 3220, 3451, 4307, 4830, 5043, 5748, 5827, 7078, 7187, 7839, 8395, 9407, 9726, 9964, 10678, 10714, 11061, 11764, 12609, 12829, 12875, 13011, 13545, 13646, 13788, 13915, 15579, 15591, 15838, 16952, 18795, 18824, 19493, 20631, 20761, 21531, 21569, 22354, 23298, 24865, 25154, 25315, 26152, 26610, 27432, 27830, 28819, 28946, 29290, 29991, 30575, 31020, 31330, 32639, 33140, 33310, 33322, 34482, 34686, 34838, 35400, 36802, 37622, 38009, 40223, 40593, 40957, 41267, 41591, 43869, 43976, 44074, 44510, 44657, 44768, 45315, 45495, 46047, 46089, 46840, 47074, 47514, 47636, 48189, 48605, 49583, 49615, 49736, 50449, 50785, 51902, 52766, 52871, 53624, 53829, 54425, 54956, 55527, 55768, 55791, 56424, 57024, 57196, 57906, 57975, 58342, 59817, 61460, 61744, 62836, 63006, 63169, 63668, 64173, 64581, 64733, 65351, 65453, 66323, 66634, 67114, 67217, 67848, 68139, 68623, 68902, 69137, 69261, 70715, 70967, 71386, 71767, 72591, 73423, 73434, 745

In [14]:
new_df2 = new_df.drop(nan_index)
new_df2
#nan 제거

,context,label
0,"[-0.1258326768875122, -0.22390116751194, -0.31...",0
1,"[-0.1259692907333374, -0.08359721302986145, -0...",1
2,"[-0.0668153241276741, -0.03492670878767967, -0...",0
3,"[-0.1706537902355194, 0.11785295605659485, -0....",0
4,"[-0.035351455211639404, 0.03357141464948654, 0...",1
...,...,...
198310,"[-0.07117301225662231, 0.08433877676725388, -0...",1
198311,"[-0.010580337606370449, 0.01028466410934925, -...",0
198312,"[-0.06006656959652901, 0.0936768501996994, -0....",0
198313,"[-0.09329956769943237, -0.05762521177530289, -...",0


In [15]:
labels2 = list(new_df2['label'])
contexts2 = list(new_df2['context'])
#로지스틱에 필요한 데이터 분리

In [16]:
from sklearn.model_selection import train_test_split

X = np.array(contexts2)
y = np.array(labels2)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [17]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)
#로지스틱 모델

LogisticRegression(class_weight='balanced')

In [18]:
print("훈련 데이터 정확도 : %f" % lgs.score(X_train, y_train))
print("테스트 데이터 정확도 : %f" % lgs.score(X_test, y_test))
#정확도

훈련 데이터 정확도 : 0.819863
테스트 데이터 정확도 : 0.820107


In [19]:
import os
import re
from konlpy.tag import Okt
okt=Okt()

In [20]:
def preprocessing(review, okt, remove_stopwords = False):
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    word_review = okt.morphs(review_text, stem = True)
    
    word_review = ' '.join(word_review)
    
    return word_review
#예측을 위한 함수

In [21]:
def get_feature_predict(words, model, num_features) :
    feature_vector = np.zeros((num_features), dtype=np.float32)
    num_words = 0
    index2word_set = set(model.wv.index2word)
    for w in words.split() :
        if w in index2word_set :
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[w])
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [22]:
def predict(review) :
    global model, num_features
    clean_review = preprocessing(review, okt, remove_stopwords=False)
    result = get_feature_predict(clean_review, model, num_features)
    List = []
    List.append(result)
    return lgs.predict(List)

In [25]:
predict("영화너무 재밌었다")

<ipython-input-21-f1669c899350>:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  index2word_set = set(model.wv.index2word)
<ipython-input-21-f1669c899350>:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  feature_vector = np.add(feature_vector, model.wv[w])


array([1])